# Неделя 5. Computer Vision
## Генеративно-состязательные сети / GAN

Ниже приведен пример генеративно-состязательной сети, реализованной в библиотеке `pytorch`.

In [1]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
from torch import device
import torch.nn as nn
import cv2
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from torchvision import datasets, transforms

In [3]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
# import torchvision.transforms as tt
# import torch
# from torch import device
# import torch.nn as nn
# import cv2
# from tqdm.notebook import tqdm
# import torch.nn.functional as F
# from torchvision.utils import save_image
# from torchvision.utils import make_grid
# import matplotlib.pyplot as plt
# import numpy as np

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
!ls

__notebook_source__.ipynb  data  generated


In [ ]:
# https://www.kaggle.com/general/74235

In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/kaggle.json /root/.kaggle

cp: cannot stat '/content/kaggle.json': No such file or directory


In [ ]:
!ls ~/.kaggle/

In [ ]:
!ls -l  cats/

ls: cannot access 'cats/': No such file or directory


In [ ]:
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


# Загрузила датасет

In [5]:
batch_size=64
latent_size=28
stats = (0.5,), (0.5,)

In [6]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
        ])),
    batch_size=batch_size, shuffle=True, drop_last=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
        ])),
    batch_size=batch_size, shuffle=True, drop_last=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [7]:
next(iter(train_loader))[1].shape

torch.Size([64])

Пишем класс генератора

In [8]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.upsample1 = nn.Sequential(
            nn.ConvTranspose2d(latent_size+10, 512, kernel_size=2, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(512),
            nn.ReLU(True)
        )
        
        self.upsample2 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True)
        )

        self.upsample3 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True)
        )

        self.upsample4 = nn.Sequential(
            nn.ConvTranspose2d(128, 28, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(28),
            nn.ReLU(True)
        )

        self.upsample5 = nn.Sequential(
            nn.ConvTranspose2d(28, 1, kernel_size=2, stride=2, padding=2, bias=False),
            nn.Tanh(),
        )

    def forward(self, x, class_vec):
        x = torch.cat([x, class_vec], 1)
        x = self.upsample1(x)
        x = self.upsample2(x)
        x = self.upsample3(x)
        x = self.upsample4(x)
        x = self.upsample5(x)
        return x
            

In [9]:
nn.functional.one_hot(next(iter(train_loader))[1].unsqueeze(1).unsqueeze(1).to(torch.int64), 10).permute(0,3,1,2).float().shape

torch.Size([64, 10, 1, 1])

In [10]:
generator1 = Generator().to(device)
# summary(generator, (28, 1, 1))

In [11]:
summary(generator1, [(28, 1, 1), (10, 1, 1)], batch_size=64)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1            [64, 512, 2, 2]          78,336
       BatchNorm2d-2            [64, 512, 2, 2]           1,024
              ReLU-3            [64, 512, 2, 2]               0
   ConvTranspose2d-4            [64, 256, 4, 4]       2,097,152
       BatchNorm2d-5            [64, 256, 4, 4]             512
              ReLU-6            [64, 256, 4, 4]               0
   ConvTranspose2d-7            [64, 128, 8, 8]         524,288
       BatchNorm2d-8            [64, 128, 8, 8]             256
              ReLU-9            [64, 128, 8, 8]               0
  ConvTranspose2d-10           [64, 28, 16, 16]          57,344
      BatchNorm2d-11           [64, 28, 16, 16]              56
             ReLU-12           [64, 28, 16, 16]               0
  ConvTranspose2d-13            [64, 1, 28, 28]             112
             Tanh-14            [64, 1,

Пишем класс Дискриминатора

In [20]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.fc1 = nn.Sequential(
            nn.Linear(10, 28*28),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Unflatten(1, (1, 28, 28))
        )
        self.conv1 = nn.Sequential(
            nn.Conv2d(2, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)            
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True) 
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 1, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Flatten(),
            nn.Sigmoid()
        )


    def forward(self, x, labels):
        c = self.fc1(labels.view(labels.size(0), -1))
        # print(c.shape)
        x = torch.cat([x, c], dim=1)
        # print(x.shape)
        x = self.conv1(x)
        #print(x.shape)
        x = self.conv2(x)
        #print(x.shape)
        x = self.conv3(x)
        #print(x.shape)
        x = self.conv4(x)
        #print(x.shape)
        return x

In [21]:
discriminator1 = Discriminator().to(device)
summary(discriminator1, [(1, 28, 28), (10, 1, 1)], batch_size=64)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [64, 784]           8,624
         LeakyReLU-2                  [64, 784]               0
         Unflatten-3            [64, 1, 28, 28]               0
            Conv2d-4           [64, 64, 14, 14]           2,048
       BatchNorm2d-5           [64, 64, 14, 14]             128
         LeakyReLU-6           [64, 64, 14, 14]               0
            Conv2d-7            [64, 128, 7, 7]         131,072
       BatchNorm2d-8            [64, 128, 7, 7]             256
         LeakyReLU-9            [64, 128, 7, 7]               0
           Conv2d-10            [64, 256, 3, 3]         524,288
      BatchNorm2d-11            [64, 256, 3, 3]             512
        LeakyReLU-12            [64, 256, 3, 3]               0
           Conv2d-13              [64, 1, 1, 1]           4,096
          Flatten-14                   

In [ ]:
discriminator1

Discriminator(
  (fc1): Sequential(
    (0): Linear(in_features=10, out_features=784, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Unflatten(dim=1, unflattened_size=(1, 28, 28))
  )
  (conv1): Sequential(
    (0): Conv2d(2, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (conv4): Sequent

Функции обучения

In [22]:
def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0]

In [35]:
sample_dir = 'generated'
os.makedirs(sample_dir, exist_ok=True)

def save_samples(index, latent_tensors, target, show=True):
    fake_images = generator1(latent_tensors, target).to(device)
    fake_fname = 'image-{0:0=4d}.png'.format(index)
    save_image(denorm(fake_images), os.path.join(sample_dir, fake_fname), nrow=8)
    print('Saving', fake_fname)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(), nrow=8).permute(1, 2, 0))

In [36]:
def train_discriminator(real_images, target, opt_d):
    # обнуляем градиенты у дискриминатора, сейчас мы будем его учить
    opt_d.zero_grad()

    # получаем предсказания дискриминатора на настоящих картинках
    real_preds = discriminator1(real_images, target).to(device) 

    # делаем вектор меток для настоящих картинок - он состоит из единиц
    # real_targets = torch.ones(real_images.size(0), 1).to(device) 
    real_targets = torch.ones(real_images.size(0), 1).to(device) 

    # считаем лосс между предсказаниями и настоящими метками
    # real_loss = torch.log(real_preds)
    real_loss = F.binary_cross_entropy(real_preds, real_targets) 

    # записываем усредненные предсказания дискриминатора
    real_score = torch.mean(real_preds).item()
    
    # создаем случайный шум: из него генератор нам что-нибудь нагенерирует
    latent = torch.randn(batch_size, latent_size, 1, 1).to(device) 

    # получаем фейковые изображения
    fake_images = generator1(latent, target).to(device)  

    # делаем вектор меток класса для фейковых изображений
    fake_targets = torch.zeros(fake_images.size(0), 1).to(device) 

    # получаем предсказания дискриминатора для фейковых изображений
    fake_preds = discriminator1(fake_images, target).to(device)  

    # считаем лосс на фейковых изображениях
    # fake_loss = torch.log(1 - fake_preds)
    fake_loss = F.binary_cross_entropy(fake_preds, fake_targets)  

    # записываем усредненные предсказания дискриминатора на фейках
    fake_score = torch.mean(fake_preds).item()

    # итоговый лосс - сумма двух лоссов
    loss = torch.mean(real_loss + fake_loss)

    # делаем обратный проход
    loss.backward()

    # обновляем веса дискриминатора
    opt_d.step()

    return loss.item(), real_score, fake_score

In [37]:
def train_generator(target, opt_g):
    # обнуляем градиенты оптимизатора генератора 
    opt_g.zero_grad()

    # генерируем случайный вектор для генератора
    latent = torch.randn(batch_size, latent_size, 1, 1).to(device)
    # latent_label = torch.randn(batch_size, 10, 1, 1).to(device)

    # получаем фейковые изображения
    fake_images = generator1(latent, target).to(device) 
    
    # получаем предсказания дискриминатора для фейковых изображений
    preds = discriminator1(fake_images, target).to(device) 

    # делаем вектор меток класса - теперь они единицы для фейковых картинок
    targets = torch.ones(batch_size, 1).to(device)

    # считаем лосс
    # loss = torch.mean(torch.log(1 - preds))
    loss = F.binary_cross_entropy(preds, targets) 
    
    # делаем обратный проход
    loss.backward()

    # обновляем веса генератора
    opt_g.step()
    
    return loss.item(), latent

In [38]:
def fit(epochs=50, lr=0.0004, start_idx=1):
    torch.cuda.empty_cache()
    
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []
  
    opt_d = torch.optim.Adam(discriminator1.to(device).parameters(), lr=lr, betas=(0.5, 0.999))
    opt_g = torch.optim.Adam(generator1.to(device).parameters(), lr=lr, betas=(0.5, 0.999))
    
    for epoch in range(epochs):
        for real_images, real_labels in tqdm(train_loader):
            
            real_images = real_images.to(device)
            target = nn.functional.one_hot(real_labels.unsqueeze(1).unsqueeze(1).to(torch.int64), 10).permute(0,3,1,2).float().to(device)
            
            loss_d, real_score, fake_score = train_discriminator(real_images, target, opt_d)
            loss_g, latent = train_generator(target, opt_g)
            
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)
        
        print(f"[{epoch+1}/{epochs}], loss_g: {loss_g:.4f}, loss_d: {loss_d:.4f}, real_score: {real_score:.4f}, fake_score: {fake_score:.4f}")
    
        save_samples(epoch+start_idx, latent, target, show=False)
    
    return losses_g, losses_d, latent, fake_scores

In [45]:
loss_g, loss_d, latent, fake_scores = fit(epochs=10, start_idx=11)

  0%|          | 0/937 [00:00<?, ?it/s]

[1/10], loss_g: 2.9742, loss_d: 0.7165, real_score: 0.6853, fake_score: 0.0550
Saving image-0011.png


  0%|          | 0/937 [00:00<?, ?it/s]

[2/10], loss_g: 4.8779, loss_d: 0.8609, real_score: 0.9074, fake_score: 0.4490
Saving image-0012.png


  0%|          | 0/937 [00:00<?, ?it/s]

...

[8/10], loss_g: 3.8726, loss_d: 0.5477, real_score: 0.8996, fake_score: 0.2823
Saving image-0018.png


  0%|          | 0/937 [00:00<?, ?it/s]

[9/10], loss_g: 1.6146, loss_d: 0.4536, real_score: 0.7738, fake_score: 0.0253
Saving image-0019.png


  0%|          | 0/937 [00:00<?, ?it/s]

[10/10], loss_g: 4.4073, loss_d: 0.2321, real_score: 0.9738, fake_score: 0.1699
Saving image-0020.png


In [33]:
!ls

data  generated  sample_data


In [34]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 14.0 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 270 kB 75.7 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 2.3 MB 53.0 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 112 kB 73.2 MB/s 
     |████████████████████████████████| 212 kB 66.8 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 80 kB 9.4 MB/s 
     |████████████████████████████████| 68 kB 6.9 MB/s 
     |████████████████████████████████| 46 kB 4.3 MB/s 
     |████████████████████████████████| 593 kB 69.6 MB/s 
     |████████████████████████████████| 856 kB 59

In [46]:
import gradio as gr

In [47]:
torch.save(generator1.state_dict(), 'MNIST_cgan.pt')

In [ ]:
!ls

data  flagged  generated  MNIST_generator.pt  sample_data


In [ ]:
# def generate(number):
#     global gen
#     gen.eval()
#     noise = torch.randn((number, 28, 1, 1))
#     tensors = gen(noise)
#     save_image(tensors, 'mnist.jpg', normalize=True)
#     return 'mnist.jpg'

# def init_model():
#     global gen 
#     gen = Generator()
#     gen.load_state_dict(torch.load('MNIST_generator.pt', map_location=torch.device('cpu')))
#     return gen

# # запуск gradio
# def run(share=True):
#     gr.Interface(
#         generate,
#         inputs=[gr.inputs.Slider(label='Number of pictures:', minimum=8, maximum=32, step=8, default=8)],
#         outputs="image",
#     ).launch(share=share)

    
# if __name__ == '__main__':
#     init_model()
#     run()

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://095e05eba2c0f68c.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [52]:
def generate(number, c):
    global gen
    gen.eval()
    noise = torch.randn(number, 28, 1,1).to('cpu')
    c = torch.tensor([c])
    target = nn.functional.one_hot(c.unsqueeze(1).unsqueeze(1).to(torch.int64), 10).permute(0,3,1,2).float().repeat(number, 1, 1, 1)
    tensors = gen(noise, target)
    save_image(tensors, '1.jpg', normalize=True)
    return '1.jpg'

# инициализация модели: архитектура + веса
def init_model():
    global gen
    gen = Generator()
    gen.load_state_dict(torch.load('MNIST_cgan.pt', map_location=torch.device('cpu')))
    return gen
# запуск gradio

def run(share=True):
    gr.Interface(
        generate,
        inputs=[gr.inputs.Slider(label='Количество цифр', minimum=1, maximum=10, step=1, default=1),
        gr.inputs.Slider(label='Число', minimum=1, maximum=10, step=1, default=1)],
        outputs = "image",
    ).launch(share=True)

if __name__ == '__main__':
    init_model()
    run()

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://f8a5255bf6230cf5.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [51]:
torch.randn(2, 50, 1, 1).to('cpu').shape

torch.Size([2, 50, 1, 1])

# 📌 Задание 2

Модифицировать GAN в Wasserstein GAN по следующей формуле: ![](https://miro.medium.com/max/1400/1*5jF5gbIDwU6k9m1ILl0Utg.jpeg)



🔖 Подсказка `keras`: [How to Develop a Wasserstein Generative Adversarial Network (WGAN) From Scratch](https://machinelearningmastery.com/how-to-code-a-wasserstein-generative-adversarial-network-wgan-from-scratch/)

🔖 Подсказка для `pyTorch`: [PyTorch-GAN](https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py)

Обрати внимание на: 
1. Метки для настоящих и сгенерированных изображений
2. Лосс-функции
3. Клиппинг весов
4. Архитектуру дискриминатора (для **WGAN** - *«критика»*)
5. Число итераций по критику и по генератору
6. Используемый оптимизатор и learning rate

# 📌 Задание 3
Помсмотри `github` с генеративными моделями и запусти произвольную из них: https://github.com/hindupuravinash/the-gan-zoo